In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from xgboost import XGBRegressor
import optuna

In [2]:
# Loading data 
X_train = pd.read_csv("../input/30-days-of-ml/train.csv").iloc[:100_000, :]
X_test = pd.read_csv("../input/30-days-of-ml/test.csv")

# Preparing data as a tabular matrix
y_train = X_train.target
X_train = X_train.set_index('id').drop('target', axis='columns')
X_test = X_test.set_index('id')

# Pointing out categorical features
categoricals = [item for item in X_train.columns if 'cat' in item]

# Dealing with categorical data using OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
X_train[categoricals] = ordinal_encoder.fit_transform(X_train[categoricals])
X_test[categoricals] = ordinal_encoder.transform(X_test[categoricals])

In [3]:
from optuna.integration import XGBoostPruningCallback

def objective(trial):
    
    params = {
            'learning_rate': trial.suggest_float("learning_rate", 0.01, 1.0, log=True),
            'reg_lambda': trial.suggest_loguniform("reg_lambda", 1e-9, 100.0),
            'reg_alpha': trial.suggest_loguniform("reg_alpha", 1e-9, 100.0),
            'subsample': trial.suggest_float("subsample", 0.1, 1.0),
            'colsample_bytree': trial.suggest_float("colsample_bytree", 0.1, 1.0),
            'max_depth': trial.suggest_int("max_depth", 1, 7),
            'min_child_weight': trial.suggest_int("min_child_weight", 1, 7),
            'gamma': trial.suggest_float("gamma", 0.1, 1.0, step=0.1)
    }

    model = XGBRegressor(
        random_state=0,
        tree_method="gpu_hist",
        predictor="gpu_predictor",
        n_estimators=10_000,
        **params,
    )
    
    model.fit(x, y, early_stopping_rounds=300, eval_set=[(x_val, y_val)], verbose=1000,
              callbacks=[XGBoostPruningCallback(trial, 'validation_0-rmse')])
    preds = model.predict(x_test)
    rmse = mean_squared_error(y_test, preds, squared=False)
    
    return rmse


In [4]:
x, x_val, y, y_val = train_test_split(X_train, y_train, random_state=0, test_size=0.2)
x, x_test, y, y_test = train_test_split(x, y, random_state=0, test_size=0.25)
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

[I 2021-10-02 20:22:11,655] A new study created in memory with name: no-name-e8f1f9b7-127e-40ae-a6ee-cab97bf770c2


[0]	validation_0-rmse:7.40017
[1000]	validation_0-rmse:0.72273
[1646]	validation_0-rmse:0.72259


[I 2021-10-02 20:22:15,187] Trial 0 finished with value: 0.7217076753069084 and parameters: {'learning_rate': 0.049310006518532, 'reg_lambda': 9.45628518237362e-09, 'reg_alpha': 2.068123919173516e-05, 'subsample': 0.8463700298755155, 'colsample_bytree': 0.7822341667593867, 'max_depth': 3, 'min_child_weight': 7, 'gamma': 0.5}. Best is trial 0 with value: 0.7217076753069084.


[0]	validation_0-rmse:3.35950
[476]	validation_0-rmse:0.72947


[I 2021-10-02 20:22:15,917] Trial 1 finished with value: 0.7249820567352396 and parameters: {'learning_rate': 0.577109653186502, 'reg_lambda': 3.822877912047026e-09, 'reg_alpha': 1.659900803891716e-09, 'subsample': 0.7187550740527774, 'colsample_bytree': 0.35617899757938987, 'max_depth': 2, 'min_child_weight': 6, 'gamma': 0.6}. Best is trial 0 with value: 0.7217076753069084.


...

[I 2021-10-02 20:22:43,032] Trial 94 pruned. Trial was pruned at iteration 0.
[I 2021-10-02 20:22:43,118] Trial 95 pruned. Trial was pruned at iteration 0.


[0]	validation_0-rmse:1.93992


[I 2021-10-02 20:22:43,224] Trial 96 pruned. Trial was pruned at iteration 5.
[I 2021-10-02 20:22:43,307] Trial 97 pruned. Trial was pruned at iteration 0.
[I 2021-10-02 20:22:43,391] Trial 98 pruned. Trial was pruned at iteration 0.


[0]	validation_0-rmse:2.67122


[I 2021-10-02 20:22:43,617] Trial 99 pruned. Trial was pruned at iteration 72.


In [5]:
print(study.best_value)
print(study.best_params)

0.7205040285992036
{'learning_rate': 0.10636257376295255, 'reg_lambda': 0.0031467926385712265, 'reg_alpha': 11.521090168065632, 'subsample': 0.9214714738628846, 'colsample_bytree': 0.2689981446216597, 'max_depth': 2, 'min_child_weight': 3, 'gamma': 0.1}
